In [0]:
# Import packages 
import numpy as np
import pandas as pd
import sys

from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
#from yellowbrick.features import RFECV

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV



import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import accuracy_score


# from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS



import logging, os, glob
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import matplotlib, pandas
from matplotlib import pyplot as plt
import numpy, sklearn, scipy, pandas
from sklearn import covariance
import numpy as np
from sklearn.impute import SimpleImputer
import statistics 
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer
import math

from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
 
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

import matplotlib
import matplotlib.pyplot as plt

In [0]:
data_file = "RDoC_MS.csv"
filename= 'RDoC_MS'


data_all = pd.read_csv(data_file)



data_all.set_index('ID', inplace=True)
ASD_labels = data_all['Group']
data_all = data_all.drop(columns = ['Group']) # remove 'Group' column. Not a feature. 
data_all_unscaled=data_all;

# Use option 1 or Option 2 for scaling data. 
# Option 1: fill NaNs with the same group's mean.  
data_all.loc[ASD_labels==1] = data_all[ASD_labels==1].fillna(data_all[ASD_labels==1].mean())
data_all.loc[ASD_labels==2] = data_all[ASD_labels==2].fillna(data_all[ASD_labels==2].mean())
data_all.loc[ASD_labels==3] = data_all[ASD_labels==3].fillna(data_all[ASD_labels==3].mean())


# Option 2: fill NaNs with the both group's mean. Option 1 should be better. 
#data_all=data_all.fillna(data_all.mean())

feature_name = data_all.columns.tolist()

y = ASD_labels.values
y= y.astype(int)



# scale_data = True  # To scale the features.
# if scale_data:
#     from sklearn.preprocessing import scale 
#     scaled_df = scale(data_all)
#     data_all = pd.DataFrame(scaled_df, columns=feature_name, index=data_all.index)

X = data_all.values


# scale_dum = 'F'
# if scale_data: scale_dum = 'T'


In [0]:
#Different X's
X_M= X[:,0:12]
X_MS= X[:,0:28]
X_S= X[:,12:28]
X_RDoC = X[:,28:]
X_RDoC_M = X[:,0:12-28:]
X_RDoC_S= X[:,12:]

XAll= [X_M,X_S,X_MS,X_RDoC,X_RDoC_M,X_RDoC_S,X]

In [0]:
print(X.shape)
numTD=sum(y==1)
print(numTD)
numASD=sum(y==2)
numTDASD=numTD+numASD
print(numTDASD)
Xbinary=X[0:numTDASD, :]
print(Xbinary.shape)
ybinary=y[0:numTDASD]
ybinary

XValues= [X_M[0:numTDASD, :],X_S[0:numTDASD, :],X_MS[0:numTDASD, :],
          X_RDoC[0:numTDASD, :],X_RDoC_M[0:numTDASD, :],X_RDoC_S[0:numTDASD, :], Xbinary]



(77, 81)
27
57
(57, 81)


In [0]:
# Test my decoder on certain feature sets: single case


# Define Cross Validation and SVM Model and Scorer 
svm= sklearn.svm.SVC(kernel='linear')
acc= []
sen= []
spe = []
f1= []


def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
scoring = {'tp': make_scorer(tp), 'tn': make_scorer(tn),
           'fp': make_scorer(fp), 'fn': make_scorer(fn)}



cvfunc_repeated = sklearn.model_selection.RepeatedStratifiedKFold(n_splits=4, n_repeats=10, random_state=8192012)
#changed 10 splits to 5


for x in XValues:

  cv_results_repeated = cross_validate(svm.fit(x, numpy.array(ybinary,dtype=int)), x, numpy.array(ybinary,dtype=int),
                          scoring=scoring, cv= cvfunc_repeated)



  tp_rep = cv_results_repeated['test_tp']
  fp_rep= cv_results_repeated['test_fp']
  tn_rep = cv_results_repeated["test_tn"]
  fn_rep = cv_results_repeated["test_fn"]
  accuracy_rep = list((tp_rep+tn_rep)/(tp_rep+tn_rep+fp_rep+fn_rep))
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))


  accuracy = statistics.mean(accuracy_rep)*100
  accuracySTD= statistics.stdev(accuracy_rep)*100
  accuracySTE = statistics.stdev(accuracy_rep)/(math.sqrt(len(accuracy_rep)))*100
  acc.append({'Accuracy': accuracy, "Accuracy STD":accuracySTD, 'Accuracy STE': accuracySTE})
  
  
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  sensitvity = statistics.mean(sensitvity_rep)*100
  sensitvitySTD = statistics.stdev(sensitvity_rep)*100 
  sensitvitySTE = (statistics.stdev(sensitvity_rep))/(math.sqrt(len(sensitvity_rep)))*100
  sen.append({"Sensitvity":sensitvity,"Sensitvity STD":sensitvitySTD, "Sensitvity STE":sensitvitySTE})

  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  specificity = statistics.mean(specificity_rep)*100
  specificitySTD = statistics.stdev(specificity_rep)*100
  specificitySTE = (statistics.stdev(specificity_rep))/(math.sqrt(len(specificity_rep)))*100
  spe.append({"Specificity":specificity,"Specificity STD":specificitySTD, "Specificity STE":specificitySTE})
  


  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))

  F1 = statistics.mean(f1_rep)*100
  F1STD = statistics.stdev(f1_rep)*100
  F1STE = (statistics.stdev(f1_rep))/(math.sqrt(len(f1_rep)))*100
  f1.append({"F1":F1,"F1 STD":F1STD,"F1STE":F1STE})

row = ["Motor","Sensory","Motor + Sensory","RDoC","RDoC + M", "RDoC + S", "RDoC+ MS"]
df1 = pd.DataFrame(acc, index= row)
df2= pd.DataFrame(sen, index= row)
df3= pd.DataFrame(spe, index= row)
df4= pd.DataFrame(f1, index= row)

print("ASD vs TD")
print(df1)
print()
print(df2)
print()
print(df3)
print()
print(df4)



                  Accuracy  Accuracy STD  Accuracy STE
Motor            89.092491      9.600423      1.517960
Sensory          90.115385      7.403807      1.170645
Motor + Sensory  95.197802      5.160619      0.815965
RDoC             94.941392      4.715022      0.745510
RDoC + M         98.411172      2.994792      0.473518
RDoC + S         96.016484      4.083978      0.645734
RDoC+ MS         96.516484      3.539904      0.559708

                 Sensitvity  Sensitvity STD  Sensitvity STE
Motor             89.910714       13.716630        2.168790
Sensory           89.955357        9.803201        1.550022
Motor + Sensory   93.571429       10.132237        1.602047
RDoC              94.776786        8.141976        1.287360
RDoC + M          97.008929        5.638049        0.891454
RDoC + S          95.758929        7.386941        1.167978
RDoC+ MS          96.339286        6.038074        0.954703

                 Specificity  Specificity STD  Specificity STE
Motor          

In [0]:
# #TD vs ASD, DCD (1,2,3)
def tn(y_true, y_pred): return sum(sum(confusion_matrix(y_true, y_pred, labels=[1,2,3])[1:3,1:3]))
def fn(y_true, y_pred): return sum(confusion_matrix(y_true, y_pred, labels=[1,2,3])[0, 1:3])
def fp(y_true, y_pred): return sum(confusion_matrix(y_true, y_pred, labels=[1,2,3])[1:3, 0:1])
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred, labels=[1,2,3])[0, 0]
scoring = {'tp': make_scorer(tp), 'tn': make_scorer(tn),
           'fp': make_scorer(fp), 'fn': make_scorer(fn)}

svm= sklearn.svm.SVC(kernel='linear')
acc= []
sen= []
spe = []
f1= []

cvfunc_repeated = sklearn.model_selection.RepeatedStratifiedKFold(n_splits=4, n_repeats=10, random_state=8192012)

for x in XAll:

  cv_results_repeated = cross_validate(svm.fit(x, numpy.array(y,dtype=int)), x, numpy.array(y,dtype=int),
                          scoring=scoring, cv= cvfunc_repeated)



  tp_rep = cv_results_repeated['test_tp']
  fp_rep= cv_results_repeated['test_fp']
  tn_rep = cv_results_repeated["test_tn"]
  fn_rep = cv_results_repeated["test_fn"]
  accuracy_rep = list((tp_rep+tn_rep)/(tp_rep+tn_rep+fp_rep+fn_rep))
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))


  accuracy = statistics.mean(accuracy_rep)*100
  accuracySTD= statistics.stdev(accuracy_rep)*100
  accuracySTE = statistics.stdev(accuracy_rep)/(math.sqrt(len(accuracy_rep)))*100
  acc.append({'Accuracy': accuracy, "Accuracy STD":accuracySTD, 'Accuracy STE': accuracySTE})
  
  
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  sensitvity = statistics.mean(sensitvity_rep)*100
  sensitvitySTD = statistics.stdev(sensitvity_rep)*100 
  sensitvitySTE = (statistics.stdev(sensitvity_rep))/(math.sqrt(len(sensitvity_rep)))*100
  sen.append({"Sensitvity":sensitvity,"Sensitvity STD":sensitvitySTD, "Sensitvity STE":sensitvitySTE})

  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  specificity = statistics.mean(specificity_rep)*100
  specificitySTD = statistics.stdev(specificity_rep)*100
  specificitySTE = (statistics.stdev(specificity_rep))/(math.sqrt(len(specificity_rep)))*100
  spe.append({"Specificity":specificity,"Specificity STD":specificitySTD, "Specificity STE":specificitySTE})
  


  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))

  F1 = statistics.mean(f1_rep)*100
  F1STD = statistics.stdev(f1_rep)*100
  F1STE = (statistics.stdev(f1_rep))/(math.sqrt(len(f1_rep)))*100
  f1.append({"F1":F1,"F1 STD":F1STD,"F1STE":F1STE})


row = ["Motor","Sensory","Motor + Sensory","RDoC","RDoC + M", "RDoC + S", "RDoC+ MS"]
df1 = pd.DataFrame(acc, index= row)
df2= pd.DataFrame(sen, index= row)
df3= pd.DataFrame(spe, index= row)
df4= pd.DataFrame(f1, index= row)
print("TD vs Rest")
print(df1)
print()
print(df2)
print()
print(df3)
print()
print(df4)

                  Accuracy  Accuracy STD  Accuracy STE
Motor            93.283626      6.912786      1.093008
Sensory          87.020468      6.080634      0.961433
Motor + Sensory  97.918129      3.064663      0.484566
RDoC             86.767544      7.637952      1.207666
RDoC + M         95.862573      3.743170      0.591847
RDoC + S         91.054825      5.161356      0.816082
RDoC+ MS         96.225146      4.108347      0.649587

                 Sensitvity  Sensitvity STD  Sensitvity STE
Motor             91.190476       11.876646        1.877863
Sensory           84.761905       13.193934        2.086144
Motor + Sensory   96.190476        6.708399        1.060691
RDoC              85.297619       14.755084        2.332984
RDoC + M          95.119048        7.152009        1.130832
RDoC + S          89.880952       10.548507        1.667865
RDoC+ MS          94.821429        8.103628        1.281296

                 Specificity  Specificity STD  Specificity STE
Motor          

In [0]:
# # # #ASD vs TD, DCD (2,1,3)
def tn(y_true, y_pred): return sum(sum(confusion_matrix(y_true, y_pred, labels=[2,1,3])[1:3, 1:3]))
def fn(y_true, y_pred): return sum(confusion_matrix(y_true, y_pred, labels=[2,1,3])[0, 1:3]) #changed from fp
def fp(y_true, y_pred): return sum(confusion_matrix(y_true, y_pred, labels=[2,1,3])[1:3, 0]) #changed from fn
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred, labels=[2,1,3])[0, 0]
scoring = {'tp': make_scorer(tp), 'tn': make_scorer(tn),
           'fp': make_scorer(fp), 'fn': make_scorer(fn)}




svm= sklearn.svm.SVC(kernel='linear')
acc= []
sen= []
spe = []
f1= []

cvfunc_repeated = sklearn.model_selection.RepeatedStratifiedKFold(n_splits=4, n_repeats=10, random_state=8192012)

for x in XAll:

  cv_results_repeated = cross_validate(svm.fit(x, numpy.array(y,dtype=int)), x, numpy.array(y,dtype=int),
                          scoring=scoring, cv= cvfunc_repeated)



  tp_rep = cv_results_repeated['test_tp']
  fp_rep= cv_results_repeated['test_fp']
  tn_rep = cv_results_repeated["test_tn"]
  fn_rep = cv_results_repeated["test_fn"]
  accuracy_rep = list((tp_rep+tn_rep)/(tp_rep+tn_rep+fp_rep+fn_rep))
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))


  accuracy = statistics.mean(accuracy_rep)*100
  accuracySTD= statistics.stdev(accuracy_rep)*100
  accuracySTE = statistics.stdev(accuracy_rep)/(math.sqrt(len(accuracy_rep)))*100
  acc.append({'Accuracy': accuracy, "Accuracy STD":accuracySTD, 'Accuracy STE': accuracySTE})
  
  
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  sensitvity = statistics.mean(sensitvity_rep)*100
  sensitvitySTD = statistics.stdev(sensitvity_rep)*100 
  sensitvitySTE = (statistics.stdev(sensitvity_rep))/(math.sqrt(len(sensitvity_rep)))*100
  sen.append({"Sensitvity":sensitvity,"Sensitvity STD":sensitvitySTD, "Sensitvity STE":sensitvitySTE})

  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  specificity = statistics.mean(specificity_rep)*100
  specificitySTD = statistics.stdev(specificity_rep)*100
  specificitySTE = (statistics.stdev(specificity_rep))/(math.sqrt(len(specificity_rep)))*100
  spe.append({"Specificity":specificity,"Specificity STD":specificitySTD, "Specificity STE":specificitySTE})
  


  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))

  F1 = statistics.mean(f1_rep)*100
  F1STD = statistics.stdev(f1_rep)*100
  F1STE = (statistics.stdev(f1_rep))/(math.sqrt(len(f1_rep)))*100
  f1.append({"F1":F1,"F1 STD":F1STD,"F1STE":F1STE})


row = ["Motor","Sensory","Motor + Sensory","RDoC","RDoC + M", "RDoC + S", "RDoC+ MS"]
df1 = pd.DataFrame(acc, index= row)
df2= pd.DataFrame(sen, index= row)
df3= pd.DataFrame(spe, index= row)
df4= pd.DataFrame(f1, index= row)
print("ASD vs Rest")
print(df1)
print()
print(df2)
print()
print(df3)
print()
print(df4)

ASD vs Rest
                  Accuracy  Accuracy STD  Accuracy STE
Motor            63.413012      8.527535      1.348322
Sensory          81.221491      9.547554      1.509601
Motor + Sensory  81.241228      7.000880      1.106936
RDoC             81.961257      7.463355      1.180060
RDoC + M         79.297515      6.284576      0.993679
RDoC + S         81.864035      6.340336      1.002495
RDoC+ MS         80.712719      6.765487      1.069717

                 Sensitvity  Sensitvity STD  Sensitvity STE
Motor             54.910714       14.157795        2.238544
Sensory           73.035714       17.264904        2.729821
Motor + Sensory   76.428571       14.252480        2.253515
RDoC              75.044643       18.612625        2.942914
RDoC + M          74.419643       15.835093        2.503748
RDoC + S          74.062500       17.421365        2.754560
RDoC+ MS          74.955357       16.831767        2.661336

                 Specificity  Specificity STD  Specificity STE
Mot

In [0]:
# # #DCD vs TD, ASD (3,1,2)
def tn(y_true, y_pred): return sum(sum(confusion_matrix(y_true, y_pred, labels=[3,1,2])[1:3, 1:3]))
def fn(y_true, y_pred): return sum(confusion_matrix(y_true, y_pred, labels=[3,1,2])[0, 1:3])
def fp(y_true, y_pred): return sum(confusion_matrix(y_true, y_pred, labels=[3,1,2])[1:3, 0:1])
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred, labels=[3,1,2])[0, 0]
scoring = {'tp': make_scorer(tp), 'tn': make_scorer(tn),
           'fp': make_scorer(fp), 'fn': make_scorer(fn)}


svm= sklearn.svm.SVC(kernel='linear')
acc= []
sen= []
spe = []
f1= []

cvfunc_repeated = sklearn.model_selection.RepeatedStratifiedKFold(n_splits=4, n_repeats=10, random_state=8192012)

for x in XAll:

  cv_results_repeated = cross_validate(svm.fit(x, numpy.array(y,dtype=int)), x, numpy.array(y,dtype=int),
                          scoring=scoring, cv= cvfunc_repeated)



  tp_rep = cv_results_repeated['test_tp']
  fp_rep= cv_results_repeated['test_fp']
  tn_rep = cv_results_repeated["test_tn"]
  fn_rep = cv_results_repeated["test_fn"]
  accuracy_rep = list((tp_rep+tn_rep)/(tp_rep+tn_rep+fp_rep+fn_rep))
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))


  accuracy = statistics.mean(accuracy_rep)*100
  accuracySTD= statistics.stdev(accuracy_rep)*100
  accuracySTE = statistics.stdev(accuracy_rep)/(math.sqrt(len(accuracy_rep)))*100
  acc.append({'Accuracy': accuracy, "Accuracy STD":accuracySTD, 'Accuracy STE': accuracySTE})
  
  
  sensitvity_rep= list((tp_rep)/(tp_rep+fn_rep))
  sensitvity = statistics.mean(sensitvity_rep)*100
  sensitvitySTD = statistics.stdev(sensitvity_rep)*100 
  sensitvitySTE = (statistics.stdev(sensitvity_rep))/(math.sqrt(len(sensitvity_rep)))*100
  sen.append({"Sensitvity":sensitvity,"Sensitvity STD":sensitvitySTD, "Sensitvity STE":sensitvitySTE})

  specificity_rep= list(tn_rep/(tn_rep+fp_rep))
  specificity = statistics.mean(specificity_rep)*100
  specificitySTD = statistics.stdev(specificity_rep)*100
  specificitySTE = (statistics.stdev(specificity_rep))/(math.sqrt(len(specificity_rep)))*100
  spe.append({"Specificity":specificity,"Specificity STD":specificitySTD, "Specificity STE":specificitySTE})
  


  f1_rep= list((2*tp_rep)/(2*tp_rep+fp_rep+fn_rep))

  F1 = statistics.mean(f1_rep)*100
  F1STD = statistics.stdev(f1_rep)*100
  F1STE = (statistics.stdev(f1_rep))/(math.sqrt(len(f1_rep)))*100
  f1.append({"F1":F1,"F1 STD":F1STD,"F1STE":F1STE})


row = ["Motor","Sensory","Motor + Sensory","RDoC","RDoC + M", "RDoC + S", "RDoC+ MS"]
df1 = pd.DataFrame(acc, index= row)
df2= pd.DataFrame(sen, index= row)
df3= pd.DataFrame(spe, index= row)
df4= pd.DataFrame(f1, index= row)


print("DCD vs Rest")
print(df1)
print()
print(df2)
print()
print(df3)
print()
print(df4)


DCD vs Rest
                  Accuracy  Accuracy STD  Accuracy STE
Motor            68.575292      8.100805      1.280850
Sensory          74.211257      9.408968      1.487689
Motor + Sensory  82.226608      7.026459      1.110981
RDoC             73.086988      8.547152      1.351423
RDoC + M         77.493421      6.580032      1.040394
RDoC + S         75.000731      7.669089      1.212589
RDoC+ MS         79.851608      6.719111      1.062385

                 Sensitvity  Sensitvity STD  Sensitvity STE
Motor                  36.0       22.735942        3.594868
Sensory                50.5       23.088459        3.650606
Motor + Sensory        66.0       21.815074        3.449266
RDoC                   46.0       22.280266        3.522819
RDoC + M               54.0       19.321836        3.055050
RDoC + S               52.5       22.043081        3.485317
RDoC+ MS               61.5       21.903050        3.463176

                 Specificity  Specificity STD  Specificity STE
Mot